# BDMLtools制作评分卡并与scorecardpy对比  <font size=3 >by 曾珂</font>

In [1]:
import scorecardpy as sc
import BDMLtools as bm
import pandas as pd

In [2]:
bm.__version__

'0.1.8'

In [3]:
sc.__version__

'0.1.9.2'

+ load germancredit data

In [4]:
dt=sc.germancredit().copy()
dt['creditability']=dt['creditability'].map({'good':0,'bad':1})

+ 将germancredit的有序category类型转换为有序数值类型

In [5]:
da=bm.dtypeAllocator().fit(dt)
dt=da.transform(dt)

In [6]:
X=dt.drop('creditability',axis=1)
y=dt['creditability']

## 制作评分卡

### 产生特征分析报告

In [7]:
breaks_list_user={'age.in.years': [26.0, 30.0, 35.0],
 'credit.amount': [4000.0, 6200.0, 8000.0],
 'credit.history': [2.0, 3.0, 4.0],
 'duration.in.month': [8.0, 16.0, 44.0],
 'foreign.worker': [1],
 'housing': [1.0],
 'installment.rate.in.percentage.of.disposable.income': [2.0, 3.0, 4.0],
 'job': [2.0, 3.0],
 'number.of.existing.credits.at.this.bank': [2.0],
 'number.of.people.being.liable.to.provide.maintenance.for': [2.0],
 'other.debtors.or.guarantors': [2.0],
 'other.installment.plans': [2.0],
 'personal.status.and.sex': [2.0, 3.0],
 'present.employment.since': [2.0, 3.0],
 'present.residence.since': [2.0],
 'property': [1.0, 2.0, 3.0],
 'purpose': ['retraining%,%car (used)',
  'radio/television',
  'furniture/equipment%,%domestic appliances%,%business',
  'repairs%,%car (new)%,%others%,%education'],
 'savings.account.and.bonds': [1.0, 2.0, 3.0],
 'status.of.existing.checking.account': [1.0, 2.0, 3.0],
 'telephone': [1.0]}

In [8]:
%%time
bin_sc=sc.woebin(dt,y='creditability',breaks_list=breaks_list_user,no_cores=1)

[INFO] creating woe binning ...
CPU times: user 526 ms, sys: 8.41 ms, total: 535 ms
Wall time: 539 ms


In [9]:
import numpy as np

In [10]:
%%time
bin_bm=bm.varReport(breaks_list_dict=breaks_list_user,n_jobs=1).fit(X,y).var_report_dict

CPU times: user 238 ms, sys: 5.38 ms, total: 243 ms
Wall time: 240 ms


### woe编码

In [11]:
%%time
dt_woe_sc = sc.woebin_ply(dt, bins=bin_sc,no_cores=1)

[INFO] converting into woe values ...
CPU times: user 296 ms, sys: 8.66 ms, total: 304 ms
Wall time: 300 ms


In [12]:
%%time
dt_woe_bm = bm.woeTransformer(varbin=bin_bm,n_jobs=1).transform(X,y)

CPU times: user 66.6 ms, sys: 2.42 ms, total: 69.1 ms
Wall time: 67.8 ms


+ 对比两份数据的编码结果

In [13]:
dt_woe_sc_1=dt_woe_sc.loc[:,dt_woe_sc.columns.str.contains('woe')]
dt_woe_sc_1.columns=[i[:-4] for i in dt_woe_sc_1.columns]
dt_woe_sc_1=dt_woe_sc_1[dt_woe_bm.columns]

+ 两者数值上几乎无区别，低精度类型条件下数值相等

In [14]:
dt_woe_sc_1.astype('float32').equals(dt_woe_bm.astype('float32')) #结果对比

True

### 回归建模

In [15]:
from sklearn.linear_model import LogisticRegression

+ 使用lr建模

In [16]:
lr_sc = LogisticRegression(penalty='l1',C=0.9,solver='saga').fit(dt_woe_sc_1, y)

In [17]:
lr_bm = LogisticRegression(penalty='l1',C=0.9,solver='saga').fit(dt_woe_bm, y)

BDMLtools的woe值转化数据dt_woe_bm(float64)与scorecardpy的woe转化数据dt_woe_sc(float64)有精度上细微的差别,,因此回归系数在会有细微差别

In [18]:
lr_sc.coef_

array([[0.61136833, 0.88740226, 0.64829701, 0.6422049 , 0.72717359,
        0.72096231, 1.65427363, 0.        , 0.        , 0.        ,
        0.96038592, 0.63854304, 0.        , 0.55413423, 0.        ,
        0.40905193, 1.01360447, 0.77094897, 0.78980351, 0.58570739]])

In [19]:
lr_bm.coef_

array([[0.61137423, 0.88738566, 0.64828619, 0.64220131, 0.72711178,
        0.72100676, 1.65428838, 0.        , 0.        , 0.        ,
        0.96039284, 0.63857966, 0.        , 0.55408399, 0.        ,
        0.40907007, 1.01362508, 0.7709723 , 0.78982361, 0.5857749 ]])

In [20]:
lr_sc.feature_names_in_[lr_bm.coef_[0]==0]

array(['job', 'number.of.existing.credits.at.this.bank',
       'number.of.people.being.liable.to.provide.maintenance.for',
       'personal.status.and.sex', 'present.residence.since'], dtype=object)

+ 制作评分卡

    这里scorecardpy产生评分卡时会去掉系数为0的特征,而bm会保留这些特征
    + 这种区别不会对后续打分带来任何影响,因为系数为0时score_point也为0

In [21]:
card_sc = sc.scorecard(bin_sc, lr_sc, dt_woe_sc_1.columns,
                           points0=600,
                           odds0=0.05263157894736842,
                           pdo=50)

In [22]:
card_obj = bm.cardScorer(lr_bm,bin_bm,
                        odds0=0.05263157894736842,
                        pdo=50,
                        points0=600).fit(X)

#bm.cardScorer可支持sklearn的LogisticRegression,也可支持statsmodels的glm-logit回归或logit回归

card_bm = card_obj.scorecard

In [23]:
print(len(card_sc),len(card_bm))

16 21


+ BDMLtools的特征分析报告文件中,任何变量都会存在missing水平,用以标示缺失值或特殊值,

    + 数据无缺失的时missing的woe为0,因此points为0,不会对打分产生任何影响
    + 若希望赋予missing值特殊的woe(比如woe较高(风险较大)),可以通过bm.cardScorer的woe_missing参数设定,这也是BMDLtools的特色
    + 更多细节请参考帮助文档

In [24]:
pd.concat(card_bm)[['points']].head(13)

points
intercept     intercept          449.0
age.in.years  [-inf, 26.0)       -23.0
              [26.0, 30.0)        -3.0
              [30.0, 35.0)        -2.0
              [35.0, inf)         14.0
              special             -0.0
              missing             -0.0
credit.amount [-inf, 4000.0)      13.0
              [4000.0, 6200.0)   -22.0
              [6200.0, 8000.0)   -25.0
              [8000.0, inf)      -65.0
              special             -0.0
              missing             -0.0

In [25]:
pd.concat(card_sc)[['variable','bin','points']].head(9)

variable              bin  points
basepoints    0      basepoints              NaN   449.0
age.in.years  55   age.in.years      [-inf,26.0)   -23.0
              56   age.in.years      [26.0,30.0)    -3.0
              57   age.in.years      [30.0,35.0)    -2.0
              58   age.in.years       [35.0,inf)    14.0
credit.amount 4   credit.amount    [-inf,4000.0)    13.0
              5   credit.amount  [4000.0,6200.0)   -22.0
              6   credit.amount  [6200.0,8000.0)   -25.0
              7   credit.amount     [8000.0,inf)   -65.0

+ 对比得分结果

In [26]:
%%time
dt_score_sc=sc.scorecard_ply(dt,card_sc)

CPU times: user 233 ms, sys: 6.71 ms, total: 239 ms
Wall time: 238 ms



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [27]:
%%time
dt_score_bm=card_obj.transform(X)

CPU times: user 65 ms, sys: 3.53 ms, total: 68.5 ms
Wall time: 65.8 ms


In [28]:
dt_score_sc['score'].equals(dt_score_bm['score'])

True